# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

# Advanced Airline AI Assistant
### original features:
1. chat with the AI assistant
2. use a Tool to get ticket price
3. generate Audio for each AI response 
### advanced features:
3. add a Tool to make a booking
4. add an Agent that translate all responses to a different language
5. add an Agent that can listen for Audio and convert to Text
6. generate audio for each user input and AI response, including both the original and translated versions

In [ ]:
# Library for language translation
 
!pip install deep_translator

In [ ]:
# Library for speech-to-text conversion
# make sure 'ffmpeg' is downloaded already

!pip install openai-whisper

In [ ]:
# Library for storing and loading audio file

pip install soundfile

In [ ]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import base64
from io import BytesIO
from IPython.display import Audio, display
import tempfile
import whisper
import soundfile as sf

In [ ]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

In [ ]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Your main responsibilities are solve customers' doubts, get ticket price and book a ticket"
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "shanghai": "$799", "wuhan": "$899"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

def book_ticket(destination_city):
    print(f"Tool book_ticket called for {destination_city}")
    city = destination_city.lower()
    global booked_cities
    if city in ticket_prices:
        price = ticket_prices.get(city, "")
        label = f"{city.title()} ({price})"
        i = booked_cities_choices.index(city.lower().capitalize())
        booked_cities_choices[i] = label
        booked_cities.append(label)
        return f"Booking confirmed for {city.title()} at {ticket_prices[city]}"
    else:
        return "City not found in ticket prices."


In [ ]:

# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_function = {
    "name": "book_ticket",
    "description": "Book a return ticket to the destination city. Call this whenever you want to book a ticket to the city, for example when the user says something like 'Book me a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to book a ticket to"
            }
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
# And this is included in a list of tools:

tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": book_function}
]

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

def talker(message):
    response = openai.audio.speech.create(
      model="tts-1",
      voice="onyx",    # Also, try replacing onyx with alloy
      input=message
    )
    
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play(audio)

In [ ]:
# This part is inspired from 'week2/community-contributions/week2_exerccise_translated_chatbot'
from deep_translator import GoogleTranslator

# Available translation language
LANGUAGES = {
    "English": "en",
    "Mandarin Chinese": "zh-CN",
    "Hindi": "hi",
    "Spanish": "es",
    "Arabic": "ar",
    "Bengali": "bn",
    "Portuguese": "pt",
    "Russian": "ru",
    "Japanese": "ja",
    "German": "de"
}

def update_lang(choice):
    global target_lang
    target_lang = LANGUAGES.get(choice, "zh-CN") 

def translate_message(text, target_lang):
    if target_lang == "en":
        return text
    try:
        translator = GoogleTranslator(source='auto', target=target_lang)
        return translator.translate(text)
    except:
        return f"Translation error: {text}"

In [ ]:
# Text-to-speech conversion
def speak(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

In [ ]:
# Update dropdown options from chatbot history
def update_options(history):
    options = [f"{msg['role']}: {msg['content']}" for msg in history]
    return gr.update(choices=options, value=options[-1] if options else "")

# Extract just the text content from selected entry
def extract_text(selected_option):
    return selected_option.split(": ", 1)[1] if ": " in selected_option else selected_option

In [ ]:
# Handles audio input as numpy array and returns updated chat history
def speak_send(audio_np, history):
    if audio_np is None:
        return history

    # Convert NumPy audio to in-memory .wav file
    sample_rate, audio_array = audio_np
    with tempfile.NamedTemporaryFile(suffix=".wav") as f:
        sf.write(f.name, audio_array, sample_rate)
        result = model.transcribe(f.name)
        text = result["text"]
        
    history += [{"role":"user", "content":text}]

    return None, history

In [ ]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if tool_name == "get_ticket_price":
        city = arguments.get("destination_city")
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city

    elif tool_name == "book_ticket":
        city = arguments.get("destination_city")
        result = book_ticket(city)
        response = {
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id            
        }
        return response, city

    else:
        return {
            "role": "tool",
            "content": f"No tool handler for {tool_name}",
            "tool_call_id": tool_call.id
        }, None

In [ ]:
# The advanced 'chat' function in 'day5'
def interact(history, translated_history):
    messages = [{"role": "system", "content": system_message}] + history
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
    reply = response.choices[0].message.content
    translated_message = translate_message(history[-1]["content"], target_lang)
    translated_reply = translate_message(reply, target_lang)
    
    history += [{"role":"assistant", "content":reply}]
    translated_history += [{"role":"user", "content":translated_message}]
    translated_history += [{"role":"assistant", "content":translated_reply}]
    
    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)

    return history, update_options(history), history, translated_history, update_options(translated_history), translated_history, gr.update(choices=booked_cities_choices, value=booked_cities)

In [ ]:
with gr.Blocks() as demo:
    target_lang = "zh-CN"
    history_state = gr.State([]) 
    translated_history_state = gr.State([])
    booked_cities_choices = [key.lower().capitalize() for key in ticket_prices.keys()]
    booked_cities = []
    model = whisper.load_model("base")

    with gr.Row():
        city_checklist = gr.CheckboxGroup(
            label="Booked Cities",
            choices=booked_cities_choices     
        )
            
    with gr.Row():
        with gr.Column():
            chatbot = gr.Chatbot(label="Chat History", type="messages")
            selected_msg = gr.Dropdown(label="Select message to speak", choices=[])
            speak_btn = gr.Button("Speak")

        with gr.Column():
            translated_chatbot = gr.Chatbot(label="Translated Chat History", type="messages")
            translated_selected_msg = gr.Dropdown(label="Select message to speak", choices=[], interactive=True)
            translated_speak_btn = gr.Button("Speak")
    
    with gr.Row():
        language_dropdown = gr.Dropdown(
                choices=list(LANGUAGES.keys()),
                value="Mandarin Chinese",
                label="Translation Language",
                interactive=True
            )
      
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    with gr.Row():
        audio_input = gr.Audio(sources="microphone", type="numpy", label="Speak with our AI Assistant:")
    with gr.Row():
        audio_submit = gr.Button("Send")
    
    def do_entry(message, history):
        history += [{"role":"user", "content":message}]
        return "", history
        
    language_dropdown.change(fn=update_lang, inputs=[language_dropdown])

    speak_btn.click(
        lambda selected: speak(extract_text(selected)),
        inputs=selected_msg,
        outputs=None
    )

    translated_speak_btn.click(
        lambda selected: speak(extract_text(selected)),
        inputs=translated_selected_msg,
        outputs=None
    )

    entry.submit(do_entry, inputs=[entry, history_state], outputs=[entry, chatbot]).then(
        interact, inputs=[chatbot, translated_chatbot], outputs=[chatbot, selected_msg, history_state, translated_chatbot, translated_selected_msg, translated_history_state, city_checklist]
    )
    
    audio_submit.click(speak_send, inputs=[audio_input, history_state], outputs=[audio_input, chatbot]).then(
        interact, inputs=[chatbot, translated_chatbot], outputs=[chatbot, selected_msg, history_state, translated_chatbot, translated_selected_msg, translated_history_state, city_checklist]
    )
    # clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

demo.launch()
